In [7]:
import pandas as pd

# -----------------------------
# Helper Functions
# -----------------------------
def cricket_overs_to_balls(overs):
    overs_int = int(overs)
    balls_part = int(round((overs - overs_int) * 10))
    return overs_int * 6 + balls_part

def balls_to_cricket_overs(balls):
    overs = balls // 6
    rem_balls = balls % 6
    return float(f"{int(overs)}.{int(rem_balls)}")

def corrected_actual_overs(row):
    adjusted_ball = row['Actual Ball']
    if row['Legal Ball'] != 'Yes':
        adjusted_ball = max(0, adjusted_ball - 1)
    if adjusted_ball == 6:
        return float(f"{int(row['Over'])}.0")
    else:
        return float(f"{int(row['Over']) - 1}.{int(adjusted_ball)}")

# -----------------------------
# Load Data
# -----------------------------
df = pd.read_csv("10JulyuptoWT20.csv", low_memory=False)
df.columns = df.columns.str.strip()
df[['Team1', 'Team2']] = df['Match'].str.split(' v ', expand=True)
df.sort_values(by=['Match', 'Date', 'Innings'], inplace=True)
df['Next_Innings'] = df.groupby(['Match', 'Date'])['Innings'].shift(-1)
mask = df['Innings'].ne(df['Next_Innings']) & (df.index < len(df))
rows_before_change = df[mask].copy().reset_index(drop=True).drop(columns=['Next_Innings'])

# -----------------------------
# Compute NRR Overs and Balls
# -----------------------------
rows_before_change['Actual Overs'] = rows_before_change.apply(corrected_actual_overs, axis=1)
rows_before_change['NRR Overs'] = rows_before_change.apply(
    lambda row: 20.0 if row['Team Wickets'] == 10 else row['Actual Overs'], axis=1
)
rows_before_change['NRR Balls'] = rows_before_change['NRR Overs'].apply(cricket_overs_to_balls)

# -----------------------------
# Team Summaries
# -----------------------------
for_summary = rows_before_change.groupby('Batting Team').agg({
    'Team Runs': 'sum',
    'NRR Balls': 'sum'
}).reset_index().rename(columns={
    'Batting Team': 'Team',
    'Team Runs': 'Runs For',
    'NRR Balls': 'NRR Balls For'
})
for_summary['Overs For'] = for_summary['NRR Balls For'].apply(balls_to_cricket_overs)
for_summary.drop(columns='NRR Balls For', inplace=True)

against_summary = rows_before_change.groupby('Bowling Team').agg({
    'Team Runs': 'sum',
    'NRR Balls': 'sum'
}).reset_index().rename(columns={
    'Bowling Team': 'Team',
    'Team Runs': 'Runs Against',
    'NRR Balls': 'NRR Balls Against'
})
against_summary['Overs Against'] = against_summary['NRR Balls Against'].apply(balls_to_cricket_overs)
against_summary.drop(columns='NRR Balls Against', inplace=True)

team_summary = pd.merge(for_summary, against_summary, on='Team', how='outer')
team_summary['NRR Balls For'] = team_summary['Overs For'].apply(cricket_overs_to_balls)
team_summary['NRR Balls Against'] = team_summary['Overs Against'].apply(cricket_overs_to_balls)
team_summary['Run Rate For'] = team_summary['Runs For'] / (team_summary['NRR Balls For'] / 6)
team_summary['Run Rate Against'] = team_summary['Runs Against'] / (team_summary['NRR Balls Against'] / 6)
team_summary['NRR'] = (team_summary['Run Rate For'] - team_summary['Run Rate Against']).round(3)
final_nrr_table = team_summary[['Team', 'Runs For', 'Overs For', 'Runs Against', 'Overs Against', 'NRR']]

# -----------------------------
# Match Results (W/L/T/NR)
# -----------------------------
match_results = []
innings_grouped = rows_before_change.groupby(['Match', 'Date'])
for (match, date), group in innings_grouped:
    if len(group) == 2:
        team1 = group.iloc[0]['Batting Team']
        team2 = group.iloc[1]['Batting Team']
        runs1 = group.iloc[0]['Team Runs']
        runs2 = group.iloc[1]['Team Runs']
        if runs1 > runs2:
            match_results.extend([
                {'Team': team1, 'W': 1, 'L': 0, 'T': 0, 'N/R': 0},
                {'Team': team2, 'W': 0, 'L': 1, 'T': 0, 'N/R': 0},
            ])
        elif runs1 == runs2:
            match_results.extend([
                {'Team': team1, 'W': 0, 'L': 0, 'T': 1, 'N/R': 0},
                {'Team': team2, 'W': 0, 'L': 0, 'T': 1, 'N/R': 0},
            ])
        else:
            match_results.extend([
                {'Team': team1, 'W': 0, 'L': 1, 'T': 0, 'N/R': 0},
                {'Team': team2, 'W': 1, 'L': 0, 'T': 0, 'N/R': 0},
            ])
    else:
        team1 = group.iloc[0]['Batting Team']
        team2 = group.iloc[0]['Bowling Team']
        match_results.extend([
            {'Team': team1, 'W': 0, 'L': 0, 'T': 0, 'N/R': 1},
            {'Team': team2, 'W': 0, 'L': 0, 'T': 0, 'N/R': 1},
        ])
results_df = pd.DataFrame(match_results)
summary_stats = results_df.groupby('Team').sum().reset_index()
summary_stats['M'] = summary_stats[['W', 'L', 'T', 'N/R']].sum(axis=1)
summary_stats['PT'] = summary_stats['W'] * 4 + summary_stats['T'] * 2 + summary_stats['N/R'] * 2

# -----------------------------
# Add No Result Games (Manual Input)
# -----------------------------
abandoned_matches = [
    ("Northamptonshire Steelbacks Women", "Worcestershire Rapids Women"),
    ("Kent Women", "Glamorgan Women")
]

for team1, team2 in abandoned_matches:
    for team in [team1, team2]:
        if team in summary_stats['Team'].values:
            summary_stats.loc[summary_stats['Team'] == team, 'M'] += 1
            summary_stats.loc[summary_stats['Team'] == team, 'N/R'] += 1
            summary_stats.loc[summary_stats['Team'] == team, 'PT'] += 2
        else:
            summary_stats = pd.concat([
                summary_stats,
                pd.DataFrame([{
                    'Team': team, 'W': 0, 'L': 0, 'T': 0, 'N/R': 1, 'M': 1, 'PT': 2
                }])
            ], ignore_index=True)

# -----------------------------
# Bonus Points Logic
# -----------------------------
innings_summary = rows_before_change[
    ['Match', 'Date', 'Innings', 'Batting Team', 'Team Runs', 'NRR Overs']
].copy()
innings_summary['Run Rate'] = innings_summary['Team Runs'] / innings_summary['NRR Overs']
valid_matches = innings_summary.groupby('Match')['Innings'].nunique()
complete_matches = valid_matches[valid_matches == 2].index
filtered = innings_summary[innings_summary['Match'].isin(complete_matches)]
match_merged = filtered.pivot(index='Match', columns='Innings', values=['Batting Team', 'Run Rate'])
match_merged.columns = ['Team1', 'Team2', 'RR1', 'RR2']
match_merged = match_merged.reset_index()
match_merged['Winner'] = match_merged.apply(
    lambda x: x['Team1'] if x['RR1'] > x['RR2'] else (x['Team2'] if x['RR2'] > x['RR1'] else 'Tie'),
    axis=1
)
match_merged['Bonus Point Team'] = match_merged.apply(
    lambda x: x['Winner'] if (
        (x['Winner'] == x['Team1'] and x['RR1'] >= 1.25 * x['RR2']) or
        (x['Winner'] == x['Team2'] and x['RR2'] >= 1.25 * x['RR1'])
    ) else None,
    axis=1
)
bonus_team_lookup = match_merged.set_index('Match')['Bonus Point Team'].to_dict()
bonus_points = pd.DataFrame({
    'Match': pd.concat([match_merged['Match'], match_merged['Match']]),
    'Team': pd.concat([match_merged['Team1'], match_merged['Team2']])
}).reset_index(drop=True)
bonus_points['BP'] = bonus_points.apply(
    lambda row: 1 if bonus_team_lookup.get(row['Match']) == row['Team'] else 0,
    axis=1
)
bonus_summary = bonus_points.groupby('Team')['BP'].sum().reset_index()

# -----------------------------
# Final Merge and Standings
# -----------------------------
final_merged = pd.merge(summary_stats, final_nrr_table, on='Team', how='outer')
final_merged = pd.merge(final_merged, bonus_summary, on='Team', how='left')
final_merged['BP'] = final_merged['BP'].fillna(0).astype(int)
final_merged['PT'] += final_merged['BP']
final_display = final_merged[
    ['Team', 'M', 'W', 'L', 'T', 'N/R', 'BP', 'PT', 'NRR', 'Runs For', 'Overs For', 'Runs Against', 'Overs Against']
].sort_values(by=['PT', 'NRR'], ascending=[False, False]).reset_index(drop=True)
final_display.index += 1

# -----------------------------
# Group Split and Export
# -----------------------------
north_group = [
    'Yorkshire Women', 'Northamptonshire Steelbacks Women',
    'Derbyshire Falcons Women', 'Leicestershire Foxes Women',
    'Worcestershire Rapids Women'
]
north_table = final_display[final_display['Team'].isin(north_group)].reset_index(drop=True)
south_table = final_display[~final_display['Team'].isin(north_group)].reset_index(drop=True)
north_table.index += 1
south_table.index += 1

north_table.to_csv("north_table.csv", index=False)
south_table.to_csv("south_table.csv", index=False)

# -----------------------------
# Display Final Tables
# -----------------------------
print("\\n📍 North Group Table:\\n", north_table)
print("\\n📍 South Group Table:\\n", south_table)

\n📍 North Group Table:\n                                 Team  M  W  L  T  N/R  BP  PT    NRR  \
1                    Yorkshire Women  5  5  0  0    0   4  24  1.954   
2         Leicestershire Foxes Women  5  3  2  0    0   1  13  0.515   
3  Northamptonshire Steelbacks Women  6  2  3  0    1   0  10 -0.764   
4           Derbyshire Falcons Women  6  2  4  0    0   1   9 -0.865   
5        Worcestershire Rapids Women  6  1  4  0    1   0   6 -0.535   

   Runs For  Overs For  Runs Against  Overs Against  
1       651       85.5           563          100.0  
2       609       97.4           572          100.0  
3       593      100.0           656           98.0  
4       658      117.1           728          112.2  
5       583      100.0           575           90.2  
\n📍 South Group Table:\n                     Team  M  W  L  T  N/R  BP  PT    NRR  Runs For  Overs For  \
1        Middlesex Women  6  5  1  0    0   1  21  0.674       831      115.1   
2        Glamorgan Women  6  4 